# Работа с Web scraping

С использованием страницы `https://yandex.ru/images/ ` сформировать запросы для поиска изображений, контент на которых соответствует классам polar bear и brown bear. Для каждого класса должно быть загружено не менее 1000 изображений. Изображения для каждого класса должны находиться в подпапке папки *dataset* с соответствующим названием. 

Не допускается: 

- Создание папок вручную. В коде должен быть отражен процесс создания папок и перемещения/загрузки в них файлов. 
- Дублирование изображений для класса. 

Примечания:
- Каждое изображение должно иметь расширение .jpg 
- Именовать файлы необходимо порядковым номером (от 0 до 999).
- Для дальнейшего удобства необходимо дополнять имя файла ведущими нулями (например, 0000, 0001, ..., 0999). Для этого необходимо использовать один из методов класса str. 
- После загрузки всех изображений, необходимо их просмотреть на соответствие классу. В случае замеченных несоответствий необходимо будет дополнить набор данных до минимального размера. Для избежания подобных ситуаций рекомендуется загружать изображения с запасом.

In [17]:
# pip install chromedriver-py==133.0.6943.53

In [ ]:
# Вариант (с кнопкой Показать еще)
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

# Параметры
image_count = 1000
dataset_dir = "dataset"

# Создаем нужные папки для сохранения изображений
def create_directory(name):
    if not os.path.exists(name):
        os.makedirs(name)

# Функция для нажатия на кнопку "Показать ещё" которая появляется во время скроллинга
def click_show_more(driver):
    try:
        show_more_button = driver.find_element(
            By.XPATH, "//button[contains(@class, 'FetchListButton') and span[text()='Показать ещё']]"
        )
        show_more_button.click()
        print("Нажата кнопка 'Показать ещё'.")
        return True
    except Exception as e:
        print("Кнопка 'Показать ещё' не найдена или не активна:", e)
        return False

# Функция для загрузки изображений
def download_images(search_query):
    create_directory(os.path.join(dataset_dir, search_query))

    # Настройка WebDriver
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')  # Не отображать окно браузера
    driver = webdriver.Chrome(options=options)

    search_url = f"https://yandex.ru/images/search?text={search_query}&noreask=1"
    driver.get(search_url)

    count = 0
    unique_images = set()  # Множество для хранения уникальных URL изображений
    last_count = 0  # Для отслеживания количества скачанных изображений
    last_time = time.time()  # Время последней загрузки изображения

    while count < image_count:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Задержка для загрузки изображений

        # Получаем HTML-код страницы
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        images = soup.find_all("img", class_="ImagesContentImage-Image ImagesContentImage-Image_clickable")

        # Удаляем дубликаты и фильтруем изображения
        for image in images:
            image_url = image.get("src")
            if image_url and not image_url.startswith("data:") and image_url not in unique_images:
                unique_images.add(image_url)
                download_image(image_url, search_query, count)  # Скачиваем изображение
                count += 1
                print(f"Current count: {count}/{image_count}")  # Вывод текущего счётчика
                last_count = count
                last_time = time.time()  # Обновляем время последней загрузки

            if count >= image_count:  # Прерываем, если достигли нужного количества изображений
                break

        # Проверка, было ли скачано изображение за последние 10 секунд
        if count == last_count and (time.time() - last_time) > 10:
            if not click_show_more(driver):
                break  # Прерываем, если кнопка не найдена

    driver.quit()  # Закрываем драйвер

def download_image(image_url, folder_name, index):
    # Добавим протокол для корректных URL
    if image_url.startswith("//"):
        image_url = "http:" + image_url

    response = requests.get(image_url)
    # Атрибут response.status_code целочисленный код ответа HTTP-статуса, например 404 (ошибка) или 200 (все ок).
    if response.status_code == 200:
        file_name = f"{str(index).zfill(4)}.jpg"
        file_path = os.path.join(dataset_dir, folder_name, file_name)
        with open(file_path, "wb") as f:
            f.write(response.content)

def main():
    create_directory(dataset_dir)  # Создаем основную папку dataset
    download_images("polar bear")   # Скачиваем изображения медведя полярного
    time.sleep(2)  # Небольшая задержка перед следующей загрузкой
    download_images("brown bear")    # Скачиваем изображения бурого медведя

if __name__ == "__main__":
    main()

Current count: 1/1000
Current count: 2/1000
Current count: 3/1000
Current count: 4/1000
Current count: 5/1000
Current count: 6/1000
Current count: 7/1000
Current count: 8/1000
Current count: 9/1000
Current count: 10/1000
Current count: 11/1000
Current count: 12/1000
Current count: 13/1000
Current count: 14/1000
Current count: 15/1000
Current count: 16/1000
Current count: 17/1000
Current count: 18/1000
Current count: 19/1000
Current count: 20/1000
Current count: 21/1000
Current count: 22/1000
Current count: 23/1000
Current count: 24/1000
Current count: 25/1000
Current count: 26/1000
Current count: 27/1000
Current count: 28/1000
Current count: 29/1000
Current count: 30/1000
Current count: 31/1000
Current count: 32/1000
Current count: 33/1000
Current count: 34/1000
Current count: 35/1000
Current count: 36/1000
Current count: 37/1000
Current count: 38/1000
Current count: 39/1000
Current count: 40/1000
Current count: 41/1000
Current count: 42/1000
Current count: 43/1000
Current count: 44/10